In [1]:

import os
import time
import json
import random
import pathlib
from collections import defaultdict
from typing import Dict, List, Tuple
from tqdm import tqdm

import numpy as np
from tqdm import trange
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

from trajdata import AgentBatch, AgentType, UnifiedDataset, SceneBatch
from trajdata.utils.batch_utils import SceneTimeBatcher
from trajdata.data_structures.scene_metadata import Scene as trajdata_Scene
from trajdata.data_structures.state import StateArray, StateTensor
from trajdata.simulation import SimulationScene, sim_metrics, sim_stats, sim_vis
from trajdata.visualization.vis import plot_agent_batch

import trajectron.evaluation as evaluation
import trajectron.visualization as vis
# from trajectron.argument_parser import args
from trajectron.model.online.online_trajectron import OnlineTrajectron
from trajectron.model.model_registrar import ModelRegistrar
from trajectron.environment import Environment, Scene, Node, DoubleHeaderNumpyArray, SceneGraph

In [2]:
arg_device = None
arg_seed = None

if not torch.cuda.is_available() or arg_device == 'cpu':
    arg_device = torch.device('cpu')
else:
    if torch.cuda.device_count() == 1:
        # If you have CUDA_VISIBLE_DEVICES set, which you should,
        # then this will prevent leftover flag arguments from
        # messing with the device allocation.
        arg_device = 'cuda:0'

    arg_device = torch.device(arg_device)

if arg_device is None:
    arg_device = 'cpu'

if arg_seed is not None:
    random.seed(arg_seed)
    np.random.seed(arg_seed)
    torch.manual_seed(arg_seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(arg_seed)

In [3]:
log_dir = '/home/abbas/Projects/trajectron/adaptive-trajectron-plus-plus/experiments/pedestrians/kf_models'
model_dir = os.path.join(log_dir, 'eth_1mode_base_tpp-10_Nov_2024_21_41_22')

# Load hyperparameters from json
conf = 'config.json'
config_file = os.path.join(model_dir, conf)
if not os.path.exists(config_file):
    raise ValueError('Config json not found!')
with open(config_file, 'r') as conf_json:
    hyperparams = json.load(conf_json)

output_save_dir = os.path.join(model_dir, 'pred_figs')
pathlib.Path(output_save_dir).mkdir(parents=True, exist_ok=True)

In [4]:
# Load evaluation environments and scenes
attention_radius = defaultdict(
    lambda: 20.0
)  # Default range is 20m unless otherwise specified.
attention_radius[(AgentType.PEDESTRIAN, AgentType.PEDESTRIAN)] = 10.0
attention_radius[(AgentType.PEDESTRIAN, AgentType.VEHICLE)] = 20.0
attention_radius[(AgentType.VEHICLE, AgentType.PEDESTRIAN)] = 20.0
attention_radius[(AgentType.VEHICLE, AgentType.VEHICLE)] = 30.0

dataset = UnifiedDataset(
    desired_data=["eupeds_eth-test_loo"],
    centric ="agent",
    history_sec=(hyperparams["history_sec"], hyperparams["history_sec"]),
    future_sec=(hyperparams["prediction_sec"], hyperparams["prediction_sec"]),
    agent_interaction_distances=attention_radius,
    incl_robot_future=hyperparams["incl_robot_node"],
    incl_raster_map=hyperparams["map_encoding"],
    only_predict=[AgentType.PEDESTRIAN],
    no_types=[AgentType.UNKNOWN],
    num_workers=hyperparams["preprocess_workers"],
    cache_location=hyperparams["trajdata_cache_dir"],
    standardize_data=False,
    data_dirs={  # Remember to change this to match your filesystem!
        "eupeds_eth": "~/Projects/trajectron/datasets/eth_ucy_peds",
    },
    verbose=True,
)

Loading data for matched scene tags: ['test_loo-eupeds_eth']


Calculating Agent Data (Serially): 100%|██████████| 1/1 [00:00<00:00, 7002.18it/s]

1 scenes in the scene index.



Structuring Agent Data Index: 100%|██████████| 1/1 [00:00<00:00, 2855.21it/s]


In [5]:
# print(dataset.envs)
# print(dataset.np_obs_type)
# print(dataset.np_state_type)
# print(dataset.obs_format)
# print(len(dataset))

In [6]:
# dataloader = DataLoader(
#     dataset,
#     batch_sampler=SceneTimeBatcher(dataset),
#     collate_fn=dataset.get_collate_fn(),
#     num_workers=4,
# )

# batch: AgentBatch
# for batch in tqdm(dataloader):
#     print(batch.scene_ts)

# print(batch)
# print(batch.agent_name)
# print(batch.agent_fut_len)
# print(batch.agent_hist_len)
# # print(batch.data_idx)
# print(batch.scene_ts)
# print(batch.curr_agent_state[0])
# print(batch.agent_hist[0][-1])
# print(batch.agent_fut[0][0])


In [7]:
# Creating a dummy environment with a single scene that contains information about the world.
# When using this code, feel free to use whichever scene index or initial timestep you wish.
scene_idx = 0

# You need to have at least acceleration, so you want 2 timesteps of prior data, e.g. [0, 1],
# so that you can immediately start incremental inference from the 3rd timestep onwards.
init_timestep = 1

sim_env_name = "eupeds_eth"
scene_name = "biwi_eth-test_loo"
desired_scene: Scene = dataset.get_scene(scene_idx)

# print(sim_scene.dataset.centric)

# obs: AgentBatch = sim_scene.reset()
# print(obs.scene_ts)

# obs = sim_scene.step()
# print(obs.scene_ts)


In [8]:
# print(type(sim_scene.scene))
# print(sim_scene.scene.agents)
# print(sim_scene.scene.agents)
# agent_ids = [item.name for item in sim_scene.scene.agents]
# print(agent_ids)

In [9]:
state = hyperparams["state"]
state_len = sum([len(state["PEDESTRIAN"][k]) for k in state["PEDESTRIAN"]])

data_header = list()
for quantity, values in state["PEDESTRIAN"].items():
    for value in values:
        data_header.append((quantity, value))

print(data_header)
# state_len
# clipped_nodes = get_clipped_nodes(obs, agent_ids, obs.curr_agent_state, hyperparams)
# for node in clipped_nodes:
#     len_list = [len(state[node.type.name][k]) for k in state[node.type.name]]
#     print(len_list)

[('position', 'x'), ('position', 'y'), ('velocity', 'x'), ('velocity', 'y'), ('acceleration', 'x'), ('acceleration', 'y'), ('heading', 'sin'), ('heading', 'cos')]


In [10]:
def get_current_data(obs):
    # get data from current observation of nodes
    # return array of size [num_agents, state_len]

    # sim_scene: SimulationScene = SimulationScene(
    #     env_name=sim_env_name,
    #     scene_name=scene_name,
    #     scene=desired_scene,
    #     dataset=dataset,
    #     init_timestep=timestep,
    #     freeze_agents=True,
    #     )
    # obs: AgentBatch = sim_scene.reset()
    # print(sim_scene.scene.length_timesteps)
    # print(sim_scene.scene.length_seconds())

    num_agents = len(obs.agent_name)
    curr_data = np.zeros((num_agents, state_len))
    for idx, agent_state in enumerate(obs.curr_agent_state):
        curr_data[idx,:-2] = agent_state[:-1].numpy()
        heading = agent_state.heading.item()
        curr_data[idx,-2] = np.sin(heading)
        curr_data[idx,-1] = np.cos(heading)
        
    # curr_data = torch.tensor(curr_data)
    # curr_data = StateTensor.from_array(curr_data, "x,y,xd,yd,xdd,ydd,s,c")
    return curr_data

# curr_data = get_current_data(obs)

In [11]:
# timestep = 1
# try:
#     sim_scene: SimulationScene = SimulationScene(
#                 env_name=sim_env_name,
#                 scene_name=scene_name,
#                 scene=desired_scene,
#                 dataset=dataset,
#                 init_timestep=timestep,
#                 freeze_agents=True,
#                 )
#     obs: AgentBatch = sim_scene.reset()
#     nodes = get_current_data(obs)
#     print(nodes)
# except ValueError as e:
#     if str(e)==f"Initial timestep {timestep} contains no agents after filtering. Please choose another initial timestep.":
#         nodes = []
#         print(nodes)
#     else:
#         raise
# except IndexError as e:
#     if str(e)==f""

In [12]:
# print('--------------------------------')
# print(obs.scene_ts)
# print('--------------------------------')
# print(obs.num_neigh)
# print('--------------------------------')
# print(obs.scene_ids)
# print('--------------------------------')
# print(obs.robot_fut_len)
# print('--------------------------------')
# print(obs.agent_types)
# print('--------------------------------')
# print(obs.agent_type)
# print('--------------------------------')
# print(obs.agent_hist_len)
# print('--------------------------------')
# print(f"current={obs.curr_agent_state[0]}")
# curr_yaw = obs.curr_agent_state[0].heading.item()
# world_from_agent = np.array(
#     [
#         [np.cos(curr_yaw), np.sin(curr_yaw)],
#         [-np.sin(curr_yaw), np.cos(curr_yaw)],
#     ]
# )
# next_state = np.zeros(7,)
# next_state[:2] = obs.agent_fut[0, 0, :2] @ world_from_agent + obs.curr_agent_state[0, :2]
# next_state[2:4] = obs.agent_fut[0, 0, 2:4] @ world_from_agent + obs.curr_agent_state[0, 2:4]
# next_state[4:6] = obs.agent_fut[0, 0, 4:6] @ world_from_agent + obs.curr_agent_state[0, 4:6]
# yaw_ac = obs.agent_fut[0, 0].heading.item()
# next_state[-1] = curr_yaw + yaw_ac
# print(next_state)
# print('--------------------------------')
# print(obs.agent_fut_len)
# print('--------------------------------')
# print(f"hist={obs.agent_hist[0]}")
# print(f"agents={obs.agent_name}")
# print(obs.dt)
# print(obs.agent_fut_len)
# print(obs.agent_fut_extent)
# print(f"future={obs.agent_fut[0, 0]}")
# fut_heading = np.arctan2 (obs.agent_fut[0, 0, -2], obs.agent_fut[0, 0, -1])
# print(fut_heading)
# print(obs.history_pad_dir)
# print(obs.neigh_hist)

# print(hyperparams["state"]["PEDESTRIAN"].items())

In [13]:
def get_clipped_nodes(timesteps):
    length = len(timesteps)
    clipped_nodes: List[Node] = list()
    for t in timesteps:
        if t<0:
            continue
        try:
            sim_scene: SimulationScene = SimulationScene(
                env_name=sim_env_name,
                scene_name=scene_name,
                scene=desired_scene,
                dataset=dataset,
                init_timestep=t,
                freeze_agents=True,
                )
            obs: AgentBatch = sim_scene.reset()
            curr_data = get_current_data(obs)
            for node in clipped_nodes:
                if node.id not in obs.agent_name:
                    node.data.data = np.vstack((node.data.data, np.full((1, state_len), np.nan)))
                else:
                    idx = obs.agent_name.index(node.id)
                    node.data.data = np.vstack((node.data.data, curr_data[idx]))
            
            for idx, name in enumerate(obs.agent_name):
                if name not in [node.id for node in clipped_nodes]:
                    node_data = DoubleHeaderNumpyArray(curr_data[idx].reshape(1, state_len), data_header)
                    clipped_nodes.append(Node(
                            node_type=AgentType(obs.agent_type[idx].item()),
                            node_id = name,
                            data=node_data,
                            first_timestep=t,
                            ))
        except ValueError as e:
            if str(e)==f"Initial timestep {t} contains no agents after filtering. Please choose another initial timestep.":
                curr_data = []
            else:
                raise
        
                
    for node in clipped_nodes:
        if node.data.data.shape[0]<length:
            len_diff = length - node.data.data.shape[0]
            node.data.data = np.vstack((np.full((len_diff, state_len), np.nan), node.data.data))
                    
    return clipped_nodes

nodes = get_clipped_nodes(list(range(-7,1)))
print(nodes)
# print(nodes[1].first_timestep)
print(nodes[0].data.data)
# print(nodes[2].data.data.shape)
# print(nodes[0].data.data.shape)


[AgentType.PEDESTRIAN/1]
[[       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan]
 [8.46000004 3.58999991 2.7750001  0.5        0.         0.
  0.17732476 0.98415239]]


In [14]:
def get_clipped_input_dict(timestep):
    input_dict: Dict[Node] = dict()
    existing_nodes = get_clipped_nodes(list(range(timestep, timestep+1)))
    for node in existing_nodes:
        input_dict[node] = node.data.data
    return input_dict

In [15]:
def create_online_env(obs : AgentBatch, init_timestep):
    # test_scene = env.scenes[scene_idx]

    online_scene = Scene(timesteps=init_timestep + 1,
                         map=None,
                         dt=obs.dt[0])
    # online_scene.nodes = test_scene.agents
    online_scene.nodes = get_clipped_nodes(
        timesteps=np.arange(init_timestep - hyperparams['maximum_history_length'],
                            init_timestep + 1))
    # online_scene.robot = test_scene.robot
    online_scene.calculate_scene_graph(attention_radius=attention_radius,
                                       edge_addition_filter=hyperparams['edge_addition_filter'],
                                       edge_removal_filter=hyperparams['edge_removal_filter'])

    env_standardization = {'PEDESTRIAN': {'position': {'x': {'mean': 0, 'std': 1}, 'y': {'mean': 0, 'std': 1}}, 'velocity': {'x': {'mean': 0, 'std': 2}, 'y': {'mean': 0, 'std': 2}}, 'acceleration': {'x': {'mean': 0, 'std': 1}, 'y': {'mean': 0, 'std': 1}}, 'heading': {'sin': {'mean': 0, 'std': 0.01}, 'cos': {'mean': 0, 'std': 0.01}}}}
    node_type_list = [AgentType(node_type.item()).name for node_type in obs.agent_type]
    return Environment(
            node_type_list=node_type_list,
            scenes=[online_scene],
            attention_radius=attention_radius,
            standardization=env_standardization
    )

In [16]:
sim_scene: SimulationScene = SimulationScene(
    env_name=sim_env_name,
    scene_name=scene_name,
    scene=desired_scene,
    dataset=dataset,
    init_timestep=init_timestep,
    freeze_agents=True,
    )
obs: AgentBatch = sim_scene.reset()

hyperparams["maximum_history_length"] = int((hyperparams["history_sec"]/sim_scene.scene.dt) + 1)
online_env = create_online_env(obs, init_timestep)

model_registrar = ModelRegistrar(model_dir, arg_device)

trajectron = OnlineTrajectron(model_registrar,
                                hyperparams,
                                arg_device)

epoch = 50
model_path = pathlib.Path(model_dir) / f'model_registrar-{epoch}.pt'
checkpoint = torch.load(model_path, map_location=arg_device)
trajectron.load_state_dict(checkpoint["model_state_dict"], strict=False)

# trajectron.set_environment(online_env, init_timestep)
trajectron.env = online_env
trajectron.scene_graph = SceneGraph(edge_radius=online_env.attention_radius)
trajectron.nodes.clear()
trajectron.node_data.clear()
trajectron.node_models_dict.clear()

# trajectron.device

for t in range(init_timestep + 1):
    trajectron.incremental_forward(
        new_inputs_dict=get_clipped_input_dict(t),
        maps=None,
        run_models=False
    )
trajectron.node_models_dict.keys()

dict_keys([AgentType.PEDESTRIAN/1])

In [17]:
for t in range(init_timestep+1, sim_scene.scene.length_timesteps):
    new_xyzh_dict: Dict[str, StateArray] = dict()
    
    start = time.time()
    dists, preds = trajectron.incremental_forward(
        new_inputs_dict=get_clipped_input_dict(t),
        maps=None,
        prediction_horizon=6,
        num_samples=1,
        full_dist=True
    )
    end = time.time()
    print("t=%d: took %.2f s (= %.2f Hz) w/ %d nodes and %d edges" % (t, end - start,
                                                                    1. / (end - start), len(trajectron.nodes),
                                                                    trajectron.scene_graph.get_num_edges()))
    
    detailed_preds_dict = dict()
    for name in obs.agent_name:
        if name in preds:
            detailed_preds_dict[name] = preds[name]

    fig, ax = plt.subplots()
    vis.visualize_distribution(ax,
                            dists)
    vis.visualize_prediction(ax,
                            {t: preds},
                            sim_scene.scene.dt,
                            hyperparams['maximum_history_length'],
                            hyperparams['prediction_horizon'])
    
    fig.savefig(os.path.join(output_save_dir, f'pred_{t}.pdf'), dpi=300)
    plt.close(fig)

TypeError: encode_total_edge_influence() missing 1 required positional argument: 'batch_size'